# Выбор локации для скважины

Добывающая компания «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходима модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проведем анализе возможной прибыли и рисков техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Для анализа предоставлены следующие данные.**

Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv. Скачать датасет

/datasets/geo_data_1.csv. Скачать датасет

/datasets/geo_data_2.csv. Скачать датасет

id — уникальный идентификатор скважины

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы)

product — объём запасов в скважине (тыс. баррелей)

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые)

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки

Бюджет на разработку скважин в регионе — 10 млрд рублей

Один баррель сырья приносит 450 рублей дохода

Интересуют регионы, в которых вероятность убытков меньше 2.5%

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

import scipy.stats as stats

import matplotlib.pyplot as plt

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
df_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [4]:
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

### Общая информация

In [5]:
# функция вывода информации о датасете
def df_info(df):
    display('общая информация с признаками')
    display(df.head())
    display('_______________________________________________________________')
    display('информация о типах данныx')
    display()
    display(df.info())
    display('_______________________________________________________________')
    display('описание')
    display()
    display(df.describe())
    display('_______________________________________________________________')
    display('пропуски')
    display()
    display(df.isna().sum())
    display('_______________________________________________________________')
    display('дубликаты')
    display()
    display(df.duplicated().sum())
    display('_______________________________________________________________')
    display('корреляция')
    display()
    display(df.corr()) 

In [6]:
df_info(df_0)

'общая информация с признаками'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'_______________________________________________________________'

'информация о типах данныx'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

'дубликаты'

0

'_______________________________________________________________'

'корреляция'

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Таблица df_0 содержат 

- 100000 объектов и 5 признаков

- Типы данных object - 'id' и float64 - 'f0', 'f1', 'f2', 'product'

- Категориальный признак: id - удален

- Пропуски типа NaN отсутствуют

- Полные дубликаты отсутствуют

- Высокая линейная зависимость между признаками отсутствует

- Требуется масштабирование признаковне 

- Кодировка категориальных признаков не требуется - их нет

In [ ]:
df_info(df_1)

**Вывод:**

1. Данные df_0, df_1, df_2 содержат 100000 объектов и 4 признака 

2. Типы данных object и float64

3. Количественные признаки: 'f0', 'f1', 'f2', 'product'

4. Категориальный признак: id - удален

5. Пропуски типа NaN отсутствуют

6. Полные дубликаты отсутствуют

7. Объекты разделены на 2 выборки: обучающую - 75%, валидационную - 20% и тестовую - 20% с присвоением переменных.

8. Высокая линейная зависимость между признаками отсутствует

9. Масштабирование признаковне требуется

10. Кодировка категориальных признаков не требуется - их нет

In [ ]:
df_info(df_2)

**Вывод:**

1. Данные df_0, df_1, df_2 содержат 100000 объектов и 4 признака 

2. Типы данных object и float64

3. Количественные признаки: 'f0', 'f1', 'f2', 'product'

4. Категориальный признак: id - удален

5. Пропуски типа NaN отсутствуют

6. Полные дубликаты отсутствуют

7. Объекты разделены на 2 выборки: обучающую - 75%, валидационную - 20% и тестовую - 20% с присвоением переменных.

8. Высокая линейная зависимость между признаками отсутствует

9. Масштабирование признаковне требуется

10. Кодировка категориальных признаков не требуется - их нет

In [ ]:
def df_hist(data):
    
    data.hist(bins = 30, figsize = (18, 6))
    plt.suptitle('Распределение признаков', size = 20)

In [ ]:
df_hist(df_0)

In [ ]:
df_hist(df_1)

In [ ]:
df_hist(df_2)

из всех датасетов удалим сважины с запасом нефти = 0

df_0 = df_0.query('product != 0')
len(df_0)

df_1 = df_1.query('product != 0')
len(df_1)

df_2 = df_2.query('product != 0')
len(df_2)

Удаление нулевых сважин пока закомментировал

Признак id на целевой влияния не оказывает - удаляем

In [ ]:
df_0 = df_0.drop('id', axis = 1)

In [ ]:
df_1 = df_1.drop('id', axis = 1)

In [ ]:
df_2 = df_2.drop('id', axis = 1)

**Вывод:**

1. Данные df_0, df_1, df_2 содержат 100000 объектов и 4 признака 

2. Типы данных object и float64

3. Количественные признаки: 'f0', 'f1', 'f2', 'product'

4. Категориальный признак: id - удален

5. Пропуски типа NaN отсутствуют

6. Полные дубликаты отсутствуют

7. Объекты разделены на 2 выборки: обучающую - 75%, валидационную - 20% и тестовую - 20% с присвоением переменных.

8. Высокая линейная зависимость между признаками отсутствует

9. Масштабирование признаковне требуется

10. Кодировка категориальных признаков не требуется - их нет

## Обучение и проверка модели для каждого региона

### Разбиение данных на обучающую и валидационную выборки в соотношении 75:25

In [ ]:
STATE = np.random.RandomState(42)

Выделение целевого признака

In [ ]:
x_0 = df_0.drop('product', axis = 1)
y_0 = df_0['product']

In [ ]:
x_1 = df_1.drop('product', axis = 1)
y_1 = df_1['product']    

In [ ]:
x_2 = df_2.drop('product', axis = 1)
y_2 = df_2['product']

Разбиение данных на обучающую и валидационную выборки в соотношении 0.75:0.25

In [ ]:
x_0_train, x_0_valid, y_0_train, y_0_valid = train_test_split(x_0, y_0, test_size=0.25, random_state=STATE)

In [ ]:
x_1_train, x_1_valid, y_1_train, y_1_valid = train_test_split(x_1, y_1, test_size=0.25, random_state=STATE)

In [ ]:
x_2_train, x_2_valid, y_2_train, y_2_valid = train_test_split(x_2, y_2, test_size=0.25, random_state=STATE)

### Обучение модели и предсказание на валидационной выборке.

инициализация модели LinearRegression

model = LinearRegression()

обучение модели на тренировочной выборке

model_0 = model.fit(x_0_train, y_0_train)
model_0

model_1 = model.fit(x_1_train, y_1_train)

model_2 = model.fit(x_2_train, y_2_train)

предсказание модели на валидационной выборке

predictions_valid_0 = model_0.predict(x_0_valid)

predictions_valid_1 = model_1.predict(x_1_valid)

predictions_valid_2 = model_2.predict(x_2_valid)

инициализация модели LinearRegression

обучение модели на тренировочной выборке

предсказание модели на валидационной выборке

In [ ]:
model_0 = LinearRegression()
model_0 = model_0.fit(x_0_train, y_0_train)
predictions_valid_0 = model_0.predict(x_0_valid)

In [ ]:
model_1 = LinearRegression()
model_1 = model_1.fit(x_1_train, y_1_train)
predictions_valid_1 = model_1.predict(x_1_valid)

In [ ]:
model_2 = LinearRegression()
model_2 = model_2.fit(x_2_train, y_2_train)
predictions_valid_2 = model_2.predict(x_2_valid)

### Предсказания и правильные ответы на валидационной выборке

In [ ]:
result_0 = pd.DataFrame(y_0_valid)
result_0['predictions_valid_0'] = predictions_valid_0
result_0

In [ ]:
result_1 = pd.DataFrame(y_1_valid)
result_1['predictions_valid_1'] = predictions_valid_1
result_1

In [ ]:
result_2 = pd.DataFrame(y_2_valid)
result_2['predictions_valid_2'] = predictions_valid_2
result_2

### Cредний запас предсказанного сырья и RMSE модели.

Функция расчета среднего запаса предсказанного сырья и RMSE модели

In [ ]:
def result_rmse(predictions_valid, y_valid):
    result_mean = predictions_valid.mean()
    rmse = mean_squared_error(y_valid,predictions_valid)**.5
    return result_mean, rmse

Cредний запас предсказанного сырья

In [ ]:
result_0_mean = result_rmse(y_0_valid,predictions_valid_0)[0]
result_0_mean

In [ ]:
result_1_mean = result_rmse(y_1_valid,predictions_valid_1)[0]
result_1_mean

In [ ]:
result_2_mean = result_rmse(y_2_valid,predictions_valid_2)[0]
result_2_mean

RMSE модели

In [ ]:
rmse_0 = result_rmse(y_0_valid,predictions_valid_0)[1]
rmse_0

In [ ]:
rmse_1 = result_rmse(y_1_valid,predictions_valid_1)[1]
rmse_1

In [ ]:
rmse_2 = result_rmse(y_2_valid,predictions_valid_2)[1]
rmse_2

In [ ]:
result_2_mean, rmse_2 = result_rmse(y_2_valid,predictions_valid_2)
result_2_mean, rmse_2

**Вывод:**


Средний запас предсказанного сырья нименьний (68.7 тыс. баррелей) в регионе "1",

в регионе "0" и регионе "2" - 92.3 и 95.1 соответственно.

Квадрат средней квадратичной ошибки у моделей для каждого региона отличается. 

Наиболее точная модель model_1 (RMSE = 0.89 тыс.бар.) - отличается сильно от остальных

далее  - model_0 (RMSE = 37.75 тыс.бар.) и model_2 (RMSE = 40.14 тыс.бар.) - точность очень низкая - ошибки более 40%

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраните в отдельных переменных.

При разведке региона исследуют 500 точек

In [ ]:
TOTAL_POINTS = 500

выбирают 200 лучших точек для разработки по данным ML

In [ ]:
BEST_TOTAL_POINTS = 200

Бюджет на разработку скважин в регионе

In [ ]:
TOTAL_BUDGET = 10_000_000_000

Бюджет на разработку одной выбранной скважины в регионе

In [ ]:
POINT_BUDGET = TOTAL_BUDGET / BEST_TOTAL_POINTS

одна тысяча баррелей сырья приносит доход

In [ ]:
PROFIT_BARR = 450_000

Нижний квантиль

In [ ]:
LOWER = .025

Верхний квантиль

In [ ]:
UPPER = .975

### Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [ ]:
value_point_min = POINT_BUDGET / PROFIT_BARR
value_point_min

Разница среднего запаса в регионе и достаточного объём сырья 

In [ ]:
result_0_mean - value_point_min

In [ ]:
result_1_mean - value_point_min

In [ ]:
result_2_mean - value_point_min

**Вывод:**


Средняя скважина во всех регионах будет убыточной.

##  Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

def revenue(target, probabilities, count):    
    pred = pd.Series(probabilities)
    pred.index = target.index  
    
    probs_sorted = pred.sort_values(ascending=False) 
    selected = target[probs_sorted.index][:count]
    return selected.sum() * PROFIT_BARR - TOTAL_BUDGET

In [ ]:
def revenue(target, probabilities, count):    
   
    probs_sorted = probabilities.sort_values(ascending=False) 
    selected = target[probs_sorted.index][:count]
    return selected.sum() * PROFIT_BARR - TOTAL_BUDGET

<div class="alert alert-info"> <b>

Устранил недочеты по индексации
    
Код сделал более емким, удалил лишние принты
    
Результаты приобрели логический вид   

</div>

**Вывод:**


В регионе "0" выбранные по предсказаниям модели (model_0) 200 скважин из 25000 дают наибольшую из 3х регионов прибыль 3_359_141_114 руб

## Риски и прибыль для каждого региона

функция расчета метрик: средней прибыли, дов. интервала и риска убытков.

def profit_info(target, probabilities, count):
    values = [] 
    target = target.reset_index(drop=True)
    
    for i in range(1000):
        
        target_subsample = target.sample(n=500, replace=True, random_state=STATE)
        probs_subsample = pd.Series(probabilities)[target_subsample.index]
        
        values.append(revenue(target_subsample, probs_subsample, count))
        
    return values

функция вывода информации о регионах

def total_info(values):
    
    values = pd.Series(values)
    lower = values.quantile(LOWER)
    upper = values.quantile(UPPER)
    risk_loss = stats.percentileofscore(values, 0)

    mean = values.mean()
    
    print('Средняя прибыль: {:_.2f}'. format(mean))
    print("2.5%-квантиль:  {:_.2f}". format(lower))
    print("97.5%-квантиль: {:_.2f}". format(upper))
    print("Риск убытков: {:.2} %". format(risk_loss))

In [ ]:
def total_info(target, probabilities, count):
            
    values = [] 
    target = target.reset_index(drop=True)
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=STATE)
        probs_subsample = pd.Series(probabilities)[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, count))
        
    values = pd.Series(values)
            
    print('Средняя прибыль: {:_.2f}'. format(pd.Series(values).mean()))
    print("2.5%-квантиль:  {:_.2f}". format(values.quantile(LOWER)))
    print("97.5%-квантиль: {:_.2f}". format(values.quantile(UPPER)))
    print("Риск убытков: {:.4} %". format(stats.percentileofscore(values, 0)))

<div class="alert alert-info"> <b>

Да, при каждо вызове profit_info, длина списка values увеличивается на 1000
    
Изменил, объявил переменную внутри функции values = [ ]
    
Объединил 2 функции в одну, return values при этом не добавляю
    
Теперь при каждом вызове total_info(target, probabilities, count), результаты немного колеблются, но их значения мне нравяться больше.

</div>

<div class="alert alert-block alert-warning">

Также в функцию revenue из функции profit_info и так передаются сириес с правильными индексами, то есть дополнительно переводить в сириес probabilities в функции revenue не нужно.

</div>


<div class="alert alert-info"> <b>
    
Устранил недочеты по индексации функции profit_info сбросом индексации target = target.reset_index(drop=True)
       
Удалил перевод в сириес в функции revenue
    
Удалил промежуточные расчеты прибыли функцией revenue

</div>

Регион "0":  средняя прибыль, 95%-й доверительный интервал и риск убытков

In [ ]:
total_info(y_0_valid, predictions_valid_0, BEST_TOTAL_POINTS)

Регион "1":  средняя прибыль, 95%-й доверительный интервал и риск убытков

In [ ]:
total_info(y_1_valid, predictions_valid_1, BEST_TOTAL_POINTS)

Регион "2":  средняя прибыль, 95%-й доверительный интервал и риск убытков

In [ ]:
total_info(y_2_valid, predictions_valid_2, BEST_TOTAL_POINTS)

**Вывод:**


Модель определения наиболее прибыльного региона и анализ рисков техникой Bootstrap, на основании представленных к анализу данных о регионах показали следующее:

1. Наибольшую среднюю прибыль имеет регион "1" 485_513_458.54 баррелей нефти    

2. Наименьшая вероятность быть убыточным менее 2.0% - регион "1"

3. Регион "1" выбран, как регион выполнивший условие - риск убытков менее 2.5% и рекомендован к разработке.

<div class="alert alert-block alert-warning">

Также для расчета рисков можно использовать такой метод `stats.percentileofscore(values, 0)`.

</div>
